In [3]:
print("OK!")

OK!


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [5]:
PINECONE_API_KEY = "pcsk_6UUUJ8_JDZjgm2w4d5PEhKzLM27ncsa6zkKoJGUJBUoi8NzfrcAtVRnAQCo6DjkQb6VDBm"
PINECONE_API_ENV = "us-east-1"

In [6]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf("C:\\Users\\LENOVO\\Desktop\\End-to-end-Medical-Chatbot-using-Llama2\\data")


In [7]:
# extracted_data

In [8]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [11]:
# text_chunks

In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
query_result

[-0.034477200359106064,
 0.031023219227790833,
 0.0067349993623793125,
 0.02610897459089756,
 -0.03936201333999634,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441470701247454,
 -0.04745051637291908,
 0.014758859761059284,
 0.07087533175945282,
 0.05552757531404495,
 0.019193289801478386,
 -0.026251329109072685,
 -0.010109508410096169,
 -0.026940548792481422,
 0.022307483479380608,
 -0.022226618602871895,
 -0.14969265460968018,
 -0.01749304309487343,
 0.007676269859075546,
 0.05435226485133171,
 0.003254482988268137,
 0.03172600269317627,
 -0.0846213772892952,
 -0.02940598875284195,
 0.051595672965049744,
 0.04812409356236458,
 -0.003314794274047017,
 -0.05827922374010086,
 0.041969284415245056,
 0.022210709750652313,
 0.1281888633966446,
 -0.022338956594467163,
 -0.011656241491436958,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122610837221146,
 -0.03117538057267666,
 0.05269957706332207,
 0.047034792602062225,
 -0.08420304954051971,
 -0.0300561785697937,
 -0.020744718

In [ ]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="genai"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index',
            dimension=1536,
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [40]:
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec

# ✅ STEP 1: Manually set credentials and index
PINECONE_API_KEY = "pcsk_6UUUJ8_JDZjgm2w4d5PEhKzLM27ncsa6zkKoJGUJBUoi8NzfrcAtVRnAQCo6DjkQb6VDBm"  # replace with your real key
PINECONE_ENV = "us-east-1"  # e.g., "gcp-starter" or "aws-us-west-2"
index_name = "genai"  # e.g., "medical-chatbot-index"

# ✅ STEP 2: Initialize Pinecone client and create index if needed
pc = Pinecone(api_key=PINECONE_API_KEY)
existing_indexes = [i.name for i in pc.list_indexes().index_list["indexes"]]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,  # depends on your embedding model
        spec=ServerlessSpec(cloud="aws", region=PINECONE_ENV)
    )
    print(f"✅ Created new index: {index_name}")
else:
    print(f"✅ Using existing index: {index_name}")

# ✅ STEP 3: Load embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ STEP 4: Upload text chunks
vectorstore = PineconeVectorStore.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name,
    namespace=""  # optional
)

print("✅ Embeddings uploaded successfully!")


✅ Using existing index: genai
✅ Embeddings uploaded successfully!


In [22]:
from langchain_pinecone import PineconeVectorStore

# STEP: Load the same embedding model
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# STEP: Load the existing index from Pinecone
docsearch = PineconeVectorStore.from_existing_index(
    index_name="genai",
    embedding=embeddings
)

# STEP: Query the index
query = "What are Allergies"
docs = docsearch.similarity_search(query, k=3)

# STEP: Print the results
for i, doc in enumerate(docs, 1):
    print(f"{i}. {doc.page_content}\n")


1. Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to foreign microor-
ganisms and particles, like pollen or dust, by producing
specific proteins called antibodies that are capable of
binding to identifying molecules, or antigens, on the
foreign organisms. This reaction between antibody and
antigen sets off a series of reactions designed to protect
the body from infection. Sometimes, this same series of

2. reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic conjunctivitis) causes redness,
irritation, and increased tearing in the eyes. Asthma caus-
es wheezing, coughing, and shortness of breath. Symp-
toms of food allergies depend on the tissues most sensi-
tive to the allergen and whether the allergen spread sys-

3. KEY TERMS
Allergen —A substance that provokes an allergic
response.
Anaphylaxis—Incr

In [23]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [24]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [30]:
from langchain.llms import CTransformers

llm = CTransformers(
    model="../model",  # 👈 go up one level to access the model folder
    model_file="llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        "max_new_tokens": 512,
        "temperature": 0.8,
    }
)


In [46]:
from langchain.schema import BaseRetriever
from typing import List, Any
from langchain_core.documents import Document
from pydantic import Field

class CustomRetriever(BaseRetriever):
    vectorstore: Any = Field()

    def get_relevant_documents(self, query: str) -> List[Document]:
        return self.vectorstore.similarity_search(query, k=2)

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return await self.vectorstore.asimilarity_search(query, k=2)

retriever = CustomRetriever(vectorstore=docsearch)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

response = qa({"query": "What are allergies?"})

print("Answer:\n", response["result"])
print("\nSources:")
for i, doc in enumerate(response["source_documents"], 1):
    print(f"{i}. {doc.page_content[:200]}...")





Answer:
 Allergies are abnormal immune system reactions to substances that are typically harmless, such as pollen, dust, mold, or certain foods. These substances can trigger a variety of symptoms, including itchy eyes, runny nose, congestion, coughing, wheezing, and difficulty breathing. In severe cases, allergies can lead to anaphylaxis, a life-threatening reaction that requires immediate medical attention.

Sources:
1. reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic con...
2. reaction. Allergic rhinitis is characterized by an itchy,
runny nose, often with a scratchy or irritated throat due
to post-nasal drip. Inflammation of the thin membrane
covering the eye (allergic con...


In [47]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin condition characterized by inflamed red bumps, blackheads, and whiteheads on the face, chest, and back. It occurs when the pores in the skin become clogged with dead skin cells, oil, and bacteria, leading to infection and inflammation. Acne can cause emotional distress and impact self-esteem, especially among teenagers and young adults. Treatment options include topical creams and gels, oral antibiotics, and hormonal therapy for more severe cases.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\LENOVO\anaconda3\envs\mchatbot\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20536\4216317267.py", line 3, in <module>
    result=qa({"query": user_input})
  File "c:\Users\LENOVO\anaconda3\envs\mchatbot\lib\site-packages\langchain\chains\base.py", line 181, in __call__
    run_id = config.get("run_id")
  File "c:\Users\LENOVO\anaconda3\envs\mchatbot\lib\site-packages\langchain\chains\base.py", line 175, in __call__
    ) -> Dict[str, Any]:
  File "c:\Users\LENOVO\anaconda3\envs\mchatbot\lib\site-packages\langchain\chains\retrieval_qa\base.py", line 131, in _call
    run_manager: Optional[CallbackManagerForChainRun] = None,
  File "c:\Users\LENOVO\anaconda3\envs\mchatbot\lib\site-packages\langchain\chains\base.py", line 320, in run
    callbacks configuration and some input/output processing.
  File 